In [2]:
print(not "0")

False


In [5]:
print(set([{"A":1}]))

TypeError: unhashable type: 'dict'

In [ ]:
import os
configs=[]#List of configs to give to PSG later
nSamples=100
atmosphereType="A1"
configFolder=r"C:\Users\Tristan\Downloads\HyPCAR3\configs"
for i,starSample in enumerate([[i]*nSamples for i in range(1,4)]):
    for counter,sample in enumerate(starSample):
        configNum=(i*nSamples)+counter+1#+1 because counter starts at 0
        configFileName=os.path.join(configFolder,f"{atmosphereType}_{configNum}.txt")
        
        print(configFileName)

In [14]:
from pyDOE import lhs
import numpy as np
gStarParamRanges = {
'starRad': (0.8, 1.3),
'starTemp': (5000, 6000),
'Kappa': (-3.5, -2.0),
'Gamma1': (-1.5,  1.1),
'Gamma2': (-1.5,  0.),
'alpha': ( 0.,   1.),
'Albedo':(0.1, 0.8),
'Distance': (1.3,15.)}
lhsSamples=lhs(len(gStarParamRanges),20000)#A list of 8 values 5 times
# scaledSamples=np.zeros_like(lhsSamples)
# print(scaledSamples)

In [1]:
import os
print(os.path.join("/home/tristanb/scratch/data", "a"))

/home/tristanb/scratch/data\a


In [2]:
print(20000%32)

0
